In [1]:
# import packages
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from scipy import stats
import statistics
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity
from scipy import optimize

In [2]:
data = pd.read_excel("/files/exercises/Homeworks/HW6/TP6.xls", skiprows = 4)
data = data.drop([0])
data = data.drop('Name', axis = 1)
data = data.astype(float)
names_company = data.columns

## Quantile estimation

In [3]:
quant_5 = np.quantile(data.iloc[:,0], 0.05)
quant_1 = np.quantile(data.iloc[:,0], 0.01)
print('Quantile 5% :', quant_5, '\nQuantile 1% :', quant_1)

Quantile 5% : -0.09334706228021589 
Quantile 1% : -0.14053384957574752


## VaR estimation

In [47]:
weights = np.repeat(0.1,10)

In [5]:
pf = pd.DataFrame(np.dot(data,weights.T), columns = ['Portfolio'])

In [6]:
means = np.ones(10)
for i in range(0,10):
    means[i] = stats.norm.fit(data.iloc[:,i].values)[0]

In [7]:
pd.DataFrame(means, index = names_company, columns = ['Estim. mean'])

,Estim. mean
AMER.EXPRESS,0.002125
AOL TIME WARNER,0.006818
AT & T,-0.001802
BLACK & DECKER,0.001037
COLGATE - PALM.,0.001985
FORD MOTOR,-0.002714
MCDONALDS,-0.000633
PFIZER,0.001445
TEXAS INSTS.,0.003713
WAL MART STORES,0.004473


In [8]:
AR1 = data.iloc[:,0].values
AR2 = data.iloc[:,1].values
AR3 = data.iloc[:,2].values
AR4 = data.iloc[:,3].values
AR5 = data.iloc[:,4].values
AR6 = data.iloc[:,5].values
AR7 = data.iloc[:,6].values
AR8 = data.iloc[:,7].values
AR9 = data.iloc[:,8].values
AR10 = data.iloc[:,9].values

cov_asset = np.cov(np.array([AR1, AR2, AR3, AR4, AR5, AR6, AR7, AR8, AR9, AR10]))
pd.DataFrame(cov_asset, index = names_company, columns = names_company)

,AMER.EXPRESS,AOL TIME WARNER,AT & T,BLACK & DECKER,COLGATE - PALM.,FORD MOTOR,MCDONALDS,PFIZER,TEXAS INSTS.,WAL MART STORES
AMER.EXPRESS,0.003577,0.002002,0.001275,0.001368,0.000972,0.001493,0.000864,0.000968,0.001806,0.001452
AOL TIME WARNER,0.002002,0.008194,0.002115,0.000926,0.000208,0.001560,0.001164,0.001167,0.002718,0.001538
AT & T,0.001275,0.002115,0.003846,0.000431,-0.000034,0.000832,0.000449,0.000317,0.001796,0.000930
BLACK & DECKER,0.001368,0.000926,0.000431,0.003101,0.000768,0.001107,0.000757,0.000343,0.000951,0.001151
COLGATE - PALM.,0.000972,0.000208,-0.000034,0.000768,0.002235,0.000434,0.000517,0.000827,-0.000140,0.000795
FORD MOTOR,0.001493,0.001560,0.000832,0.001107,0.000434,0.002637,0.000662,0.000491,0.001297,0.000983
MCDONALDS,0.000864,0.001164,0.000449,0.000757,0.000517,0.000662,0.002075,0.000531,0.000690,0.000546
PFIZER,0.000968,0.001167,0.000317,0.000343,0.000827,0.000491,0.000531,0.002227,0.000341,0.000761
TEXAS INSTS.,0.001806,0.002718,0.001796,0.000951,-0.000140,0.001297,0.000690,0.000341,0.006745,0.000842
WAL MART STORES,0.001452,0.001538,0.000930,0.001151,0.000795,0.000983,0.000546,0.000761,0.000842,0.002521


In [9]:
print('Skewness: ', stats.skew(pf.values)[0], '\nKurtosis: ', stats.kurtosis(pf.values)[0])

Skewness:  0.06314676534352914 
Kurtosis:  1.1646283780273032


In [10]:
- np.dot(weights, means.T) - math.sqrt(np.dot(np.dot(weights, cov_asset.T), weights.T))*stats.norm.ppf(0.05)

0.05606581682634763

In [11]:
def gaussianVaR(VaR, obs, bandwidth, alpha):
    return (np.mean(stats.norm.cdf((-VaR - obs)/bandwidth))-alpha)**2

In [12]:
obs = np.reshape(pf.values, 243)
bandwidth = pow(len(pf), -0.2) * pow(statistics.variance(np.reshape(pf.values, 243)), 0.5)
alpha = 0.05
guesses = -0.2

In [13]:
estimates_Var = optimize.fmin(func = gaussianVaR, x0 = guesses, args = (obs, bandwidth, alpha,))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 34


In [14]:
estimates_Var[0]

0.0580468749999995

## Sensitivities of VaR

In [15]:
- means - np.dot(cov_asset, weights.T) / math.sqrt(np.dot(np.dot(weights, cov_asset.T), weights.T)) * stats.norm.ppf(0.05)

array([0.07183328, 0.09440863, 0.05786043, 0.05008028, 0.02887156,
       0.05660511, 0.03933869, 0.03592766, 0.07620257, 0.04952996])

In [16]:
def gaussiankernel(obs_x, obs_y, VaR_hat, bandwidth = None):
    """
    kernel_pdf = gaussiankernel(obs,nb_column,bandwidth)
    
    The function gaussiankernel estimates the density of the distribution underlying the sample with the non-parametric gaussian kernel method.  
    
    Remark:
    - The density is estimated between the min and the max of the sample of observations, for nb_points equidistant points.

    Parameters
    ----------
    obs : array
        a vector of observations 
    nb_column : int
        the number of points where the density is estimated (see the remark) 
    bandwidth : double
        the length of the window (optional argument)
    
    Returns
    -------
    kernel_pdf
        a matrix (2, nb_points) where the first row contains the estimation points and the second row contains the values of the density. 

    """

    # the following calculations require obs to be an one-column array.
    if not isinstance(obs_x, np.ndarray):
        print('Error: please transfer the input variable to an array first!')
    if not isinstance(obs_y, np.ndarray):
        print('Error: please transfer the input variable to an array first!')
    #if obs_x.ndim != 1:
    #    print('Error: please transfer the input variable to an one-dimensional array!')

    nb_obs = len(obs_x)
    nb_column = obs_y.shape[1]
    
    # The 3rd input argument of the function gaussiankernel is optional.
    # If no value is passed for the bandwith, the program will a "rule of thumb" value: (https://en.wikipedia.org/wiki/Kernel_density_estimation)
    if bandwidth is None:
        bandwidth = pow(nb_obs, -0.2) * pow(statistics.variance(obs_x.reshape(243)), 0.5)
        

    # We calculate all the points in one step by using the functions mean and the kronecker product.
    # This is the most compact and efficient writing as it optimizes the use of the preprogrammed functions. You should "vectorize" the code at best to obtain a good performance.
    # Check the help on different functions for more details.

    x = (np.outer(obs_x, np.ones(nb_column)) + VaR_hat)/bandwidth
    k = np.exp(-0.5*np.power(x, 2))/np.power(2*math.pi,0.5)
    y = obs_y
    f_above = np.mean((-y*k), axis = 0)/bandwidth
    f_below = np.mean(k, axis=0)/bandwidth
    f = f_above/f_below
    kernel_pdf = f
    
    # Slow way: 
#     def calc_kernel(obs, x, bandwidth):
#        return mean(np.exp((-0.5)*np.power((obs - x)/bandwidth, 2)))/bandwidth/np.power(2*pi,0.5)
#     f = np.array([calc_kernel(obs, x, bandwidth) for x in kernel_pdf])

    return kernel_pdf

In [17]:
obs_x = pf.values
obs_y = data.values
VaR_hat = estimates_Var

In [18]:
gaussiankernel(obs_x, obs_y, VaR_hat)

array([0.06637222, 0.08788811, 0.05978446, 0.04286349, 0.02365767,
       0.04832397, 0.03919093, 0.0383557 , 0.0557471 , 0.04867303])

## Comparison with mean-variance optimization

In [19]:
def min_var(weight, cov_asset, obj, means):
    return np.dot(np.dot(weight, cov_asset.T), weight.T)

#def min_VaR(weight, cov_asset, obj, means):
#    return (np.mean(stats.norm.cdf((-VaR - obs)/bandwidth))-alpha)**2

def constr(weight):
    return sum(weight) - 1

def constr1(weight):
    return np.dot(weight, means.T) - obj  

In [20]:
guesses = np.repeat(0.1,10)
obj = 0.05
bnds = ((0,1),)*10
cons = [{'type':'eq', 'fun': constr},
        {'type':'eq', 'fun': constr1}]

opt_weight = optimize.minimize(min_var, x0 = guesses, method = 'SLSQP', constraints=cons, args = (cov_asset, means, obj,))

In [21]:
opt_weight.x

array([ 0.50372701,  1.66772605, -2.01405417,  0.06323794,  1.26027876,
       -3.81231889, -1.33433282, -0.06352053,  1.16797902,  3.56127762])

In [22]:
def min_VaR(VaR, data, weights, bandwidth, means):
    obs = np.dot(data,weights.T)
    return (np.mean(stats.norm.cdf((-VaR - obs)/bandwidth))-0.05)**2

In [23]:
data = data
bandwidth = pow(len(data), -0.2) * pow(statistics.variance(obs_x.reshape(243)), 0.5)
guesses = np.repeat(0.1,10)
obj = 0.05
bnds = ((0,1),)*10
cons = [{'type':'eq', 'fun': constr},
        {'type':'eq', 'fun': constr1}]

opt_weight2 = optimize.minimize(min_VaR, x0 = guesses, method = 'SLSQP', constraints = cons, bounds = bnds, args = (data, means, obj, bandwidth,))

ValueError: operands could not be broadcast together with shapes (10,) (243,) 

In [ ]:
means.dtype

In [ ]:
means.T